In [2]:
import functions as mf
import pandas as pd


In [3]:
df_demo, df_clients, df_web1, df_web2 = mf.load_data()


----------------------

                    -- Concating and merging 3 files to df_final_web_data

----------------------

In [4]:
df_web_experiment = mf.merge_data(df_demo, df_clients, df_web1, df_web2)


----------------------

                    -- Cleaning and sorting df_web_experiment

----------------------

In [5]:
df_web_experiment = mf.clean_web_data(df_web_experiment)


/Users/muayadhilamia/Desktop/Ironhack/Week-5/Project/week5_6_project/notebooks/functions.py:43: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["process_step"] = df["process_step"].replace({


----------------------

                    -- Adding new columns by group by to df_web_experiment

----------------------

In [6]:
df_web_experiment = mf.add_features(df_web_experiment)


----------------------

                    -- Merging all of them together

----------------------

In [7]:
df_raw = mf.merge_all(df_web_experiment, df_demo)


----------------------

                    -- Cleaning df_raw

----------------

In [8]:
df_raw = mf.clean_final(df_raw)


In [ ]:
df_raw

In [8]:
# df_raw.to_csv("df_cleand_raw_m.csv")